# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [8]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    for file_path in file_path_list:
        print(file_path)

/workspace/home
/workspace/home/event_data/2018-11-10-events.csv
/workspace/home/event_data/2018-11-20-events.csv
/workspace/home/event_data/2018-11-09-events.csv
/workspace/home/event_data/2018-11-11-events.csv
/workspace/home/event_data/2018-11-07-events.csv
/workspace/home/event_data/2018-11-27-events.csv
/workspace/home/event_data/2018-11-21-events.csv
/workspace/home/event_data/2018-11-15-events.csv
/workspace/home/event_data/2018-11-17-events.csv
/workspace/home/event_data/2018-11-06-events.csv
/workspace/home/event_data/2018-11-24-events.csv
/workspace/home/event_data/2018-11-23-events.csv
/workspace/home/event_data/2018-11-04-events.csv
/workspace/home/event_data/2018-11-30-events.csv
/workspace/home/event_data/2018-11-29-events.csv
/workspace/home/event_data/2018-11-01-events.csv
/workspace/home/event_data/2018-11-16-events.csv
/workspace/home/event_data/2018-11-02-events.csv
/workspace/home/event_data/2018-11-13-events.csv
/workspace/home/event_data/2018-11-08-events.csv
/wor

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [79]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [11]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [12]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [15]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS nhannv13
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)


#### Set Keyspace

In [16]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('nhannv13')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [22]:
# create table artist_song_for_session
try:
    query = """ CREATE TABLE IF NOT EXISTS artist_song_for_session 
                (sessionId int, 
                itemInSession int, 
                artist text, 
                song text, 
                length float, 
                PRIMARY KEY (sessionId, itemInSession))"""
    session.execute(query)
except Exception as e:
    print("Err: create table ...")
    print(e)

In [23]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO artist_song_for_session 
                    (sessionId, itemInSession, artist, song, length)"""
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [41]:
query = """
    SELECT artist, song, length
    FROM artist_song_for_session
    WHERE sessionId = 338 AND itemInSession = 4;
"""
try:
    result = session.execute(query)
    if result:
        for row in result:
            print(row.artist, row.song, row.length)
    else:
        print("No results found.")
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [40]:
# Add in the SELECT statement to verify the data was entered into the table
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """
    SELECT artist, song, length
    FROM artist_song_for_session
    WHERE sessionId = 338 AND itemInSession = 4;
"""
try:
    result = session.execute(query)
    print("-" * 90)
    print("| {:<20} | {:<31} | {:<30} |".format("Artist", "Song", "Length"))
    print("-" * 90)
    if result:
        for row in result:
            print("| {:<20} | {:<31} | {:<30} |".format(row.artist, row.song, row.length))
        print("-" * 90)
    else:
        print("No results found.")
except Exception as e:
    print(e)

------------------------------------------------------------------------------------------
| Artist               | Song                            | Length                         |
------------------------------------------------------------------------------------------
| Faithless            | Music Matters (Mark Knight Dub) | 495.30731201171875             |
------------------------------------------------------------------------------------------


In [42]:
# create table artist_song_user_for_user_session
try:
    query = """ CREATE TABLE IF NOT EXISTS artist_song_user_for_user_session 
                (userId int, 
                sessionId int, 
                itemInSession int, 
                artist text, 
                song text, 
                firstName text, 
                lastName text, 
                PRIMARY KEY ((userId, sessionId), itemInSession))"""
    session.execute(query)
except Exception as e:
    print("Err: create table ...")
    print(e)

In [43]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO artist_song_user_for_user_session 
                (userId, sessionId, artist, song, itemInSession, firstName, lastName)"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

In [71]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """
        SELECT artist, song, firstName, lastName
        FROM artist_song_user_for_user_session
        WHERE userId = 10 AND sessionId = 182
        ORDER BY itemInSession; """
try:
    result = session.execute(query)
    print("-" * 106)
    print("| {:<18} | {:<55} | {:<10} | {:<10} |".format("Artist", "Song", "First name", "Last name"))
    print("-" * 106)
    if result:
        for row in result:
            print("| {:<18} | {:<55} | {:<10} | {:<10} |".format(row.artist, row.song, row.firstname, row.lastname))
        print("-" * 106)
    else:
        print("No results found.")
except Exception as e:
    print(e)
                    

----------------------------------------------------------------------------------------------------------
| Artist             | Song                                                    | First name | Last name  |
----------------------------------------------------------------------------------------------------------
| Down To The Bone   | Keep On Keepin' On                                      | Sylvie     | Cruz       |
| Three Drives       | Greece 2000                                             | Sylvie     | Cruz       |
| Sebastien Tellier  | Kilometer                                               | Sylvie     | Cruz       |
| Lonnie Gordon      | Catch You Baby (Steve Pitron & Max Sanna Radio Edit)    | Sylvie     | Cruz       |
----------------------------------------------------------------------------------------------------------


In [77]:
# create table user_for_song
try:
    query = """ CREATE TABLE IF NOT EXISTS user_for_song 
                (song text, 
                userId int,
                firstName text, 
                lastName text, 
                PRIMARY KEY (song, userId))"""
    session.execute(query)
except Exception as e:
    print("Err: create table ...")
    print(e)

In [78]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_for_song 
                (song, userId, firstName, lastName)"""
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [84]:
query = """
    SELECT firstName, lastName
    FROM user_for_song
    WHERE song = 'All Hands Against His Own';
"""
try:
    result = session.execute(query)
    print("-" * 47)
    print("| {:<20} | {:<20} |".format("First name", "Last name"))
    print("-" * 47)
    if result:
        for row in result:
            print("| {:<20} | {:<20} |".format(row.firstname, row.lastname))
        print("-" * 47)
    else:
        print("No results found.")
except Exception as e:
    print(e)


-----------------------------------------------
| First name           | Last name            |
-----------------------------------------------
| Jacqueline           | Lynch                |
| Tegan                | Levine               |
| Sara                 | Johnson              |
-----------------------------------------------


### Drop the tables before closing out the sessions

In [85]:
## Drop the table before closing out the sessions
try:
    session.execute("drop table artist_song_for_session")
    session.execute("drop table artist_song_user_for_user_session")
    session.execute("drop table user_for_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [86]:
session.shutdown()
cluster.shutdown()

In [88]:
!jupyter nbconvert --to html *.ipynb

[NbConvertApp] Converting notebook Project_1B_ Project_Template.ipynb to html
[NbConvertApp] Writing 327723 bytes to Project_1B_ Project_Template.html
